<a href="https://colab.research.google.com/github/ldrouvot/Risk-Detection-in-Construction-Industry/blob/main/Risk_detection__construction_industry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet de détection des accidents de travail dans un contexte BTP

# 💻Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Stage et projet/projet/projet deep learning/Notebook

/content/drive/MyDrive/Stage et projet/projet/projet deep learning/Notebook


In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from datetime import datetime
import json
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist

ModuleNotFoundError: No module named 'missingno'

In [ ]:
!pip install pandas openpyxl

In [ ]:
# excel_file_1 = ("/content/drive/MyDrive/Stage et projet/projet/projet deep learning/Données/Datas_20231207.xlsx")
excel_file_1 = ("/content/drive/MyDrive/projet deep learning/Données/Datas_20231207.xlsx")
xls_1 = pd.ExcelFile(excel_file_1)

In [ ]:
# first excel
sheet_names = xls_1.sheet_names
dict_df = {}
for sheet_name in sheet_names:
    dict_df[sheet_name] = xls_1.parse(sheet_name)
for sheet_name, df in dict_df.items():
    print(f"Nom de la feuille : {sheet_name}")
    # print(df)


In [ ]:
dict_df.keys()


#💹Fonctions

In [ ]:
def missing_percentage(df, figsize=(24, 6)):
  """A function for plotting missing ratios."""

  total = df.isnull().sum().sort_values(
      ascending=False)[df.isnull().sum().sort_values(ascending=False) != 0]
  percent = (df.isnull().sum().sort_values(ascending=False) / len(df) *
              100)[(df.isnull().sum().sort_values(ascending=False) / len(df) *
                    100) != 0]
  missing = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

  fig, ax = plt.subplots(figsize=figsize)
  sns.barplot(x=missing.index, y='Percent', data=missing, palette='coolwarm_r')
  plt.xticks(rotation=90)

  display(missing.T.style.background_gradient(cmap='Reds', axis=1))

def calcul_annee(x):
   return ((datetime.today()-x).days)/365


def plot_scatter_with_regression(df, x_feature, y_feature, xlim=200, ylim=100000, figsize=(15, 15)):
    """
    Plot a scatter plot with regression line for two features from a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the features.
    - x_feature (str): The feature for the x-axis.
    - y_feature (str): The feature for the y-axis.
    - xlim (int): The upper limit for x-axis values to be displayed. Default is 200.
    - ylim (int): The upper limit for y-axis values to be displayed. Default is 100000.
    - figsize (tuple): Size of the resulting plot.

    Returns:
    - None: Displays a scatter plot with regression line.
    """

    # Set font scale for better readability
    sns.set(font_scale=1.1)

    # Filter the dataframe based on provided limits for x and y features
    filtered_df = df[(df[x_feature] < xlim) & (df[y_feature] < ylim)]

    # Create the scatter plot with regression line
    sns.lmplot(data=filtered_df, x=x_feature, y=y_feature, line_kws={'color': 'red'})

    # Set plot labels and title
    plt.xlabel(x_feature)
    plt.ylabel(y_feature)
    plt.title(f"Scatter Plot of {x_feature} and {y_feature} with Regression Line")

    # Display the plot
    plt.show()

def plot_correlation_with_feature_v2(df, target_feature, threshold=.3, figsize=(10, 8), cmap='coolwarm'):
    """
    Plot the correlation of a target feature with other numeric features in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing numeric features.
    - target_feature (str): The feature for which correlations with other features are to be plotted.
    - figsize (tuple): Size of the resulting plot.

    Returns:
    - None: Displays a horizontal bar plot of correlations.
    """

    # Calculate the correlation between the target feature and other numeric features in the DataFrame
    feature_corr = df.corr()[target_feature]

    # Drop the correlation of the target feature with itself to avoid redundancy
    feature_corr = feature_corr.drop(target_feature)

    # Take the absolute values of the correlation ratios for better comparison
    feature_corr_abs = feature_corr.abs()

    # Sort the correlation values in descending order based on their absolute values
    feature_corr_sorted = feature_corr_abs.sort_values(ascending=False)

    # Define custom color palettes (diverging and sequential palettes are defined but only the diverging one is used in this case)
    cmap = sns.color_palette('coolwarm_r', n_colors=len(feature_corr_sorted))

    # Set font scale for better readability
    sns.set(font_scale=1.1)

    # Create a new figure with the provided figsize
    plt.figure(figsize=figsize)

    # Create a horizontal bar plot displaying the correlation of the target feature with other features
    sns.barplot(x=feature_corr_sorted.values, y=feature_corr_sorted.index, palette=cmap)

    # Set plot labels and title
    plt.xlabel(f"Absolute Correlation with {target_feature}")
    plt.ylabel("Features")
    plt.title(f"Correlation of {target_feature} with other Features (Sorted by Absolute Correlation)")

    # Display the plot
    plt.show()

#Création des dataframes et de leurs copies


In [ ]:
df_acc = dict_df['Acc Trav']
df_acc_c = pd.DataFrame(df_acc)
df_donnee = dict_df['Données personnelles']
df_donnee_c = pd.DataFrame(df_donnee)
df_meteo = dict_df['Météo']
df_meteo_c = pd.DataFrame(df_meteo)
df_abs = dict_df['Absences']
df_abs_c = pd.DataFrame(df_abs)
df_me = dict_df['Messages envoyés']
df_me_c = pd.DataFrame(df_me)
df_mr = dict_df['Messages reçus']
df_mr_c = pd.DataFrame(df_mr)
df_evenements = dict_df['Evenements']
df_evenements_c = pd.DataFrame(df_evenements)
df_dmd = dict_df['Demandes']
df_dmd_c = pd.DataFrame(df_dmd)
df_hab = dict_df['Habilitations']
df_hab_c = pd.DataFrame(df_hab)
df_formations = dict_df['Formations']
df_formations_c = pd.DataFrame(df_formations)


# Encoding

In [ ]:
#encoding de l'accident de travail
df_acc_c['AT reconnu?'] = df_acc_c['AT reconnu?'].replace("oui", 1)
df_acc_c['AT reconnu?']= df_acc_c['AT reconnu?'].replace("non", 0)

#encoding messages envoyés
df_me_c['Type prévention ?'] = df_me_c['Type prévention ?'].replace("oui", 1)
df_me_c['Type prévention ?']= df_me_c['Type prévention ?'].replace("non", 0)

df_me_c['Positif ?'] = df_me_c['Positif ?'].replace("oui", 1)
df_me_c['Positif ?']= df_me_c['Positif ?'].replace("non", 0)

#encoding messages recus
df_mr_c['Type prévention ?'] = df_mr_c['Type prévention ?'].replace("oui", 1)
df_mr_c['Type prévention ?']= df_mr_c['Type prévention ?'].replace("non", 0)

df_mr_c['Positif ?'] = df_mr_c['Positif ?'].replace("oui", 1)
df_mr_c['Positif ?']= df_mr_c['Positif ?'].replace("non", 0)

#encoding données personnelles
df_donnee_c['Civilité'] = df_donnee_c['Civilité'].replace("Monsieur", 0)
df_donnee_c['Civilité'] = df_donnee_c['Civilité'].replace("Madame", 1)
df_donnee_c['Type de contrat'].replace("Contrat d'apprentissage entreprises non artisanales de plus de 10 salariés (loi de 1987)", 'CAP', inplace=True,)
df_donnee_c['Type de contrat'].replace("Contrat d'apprentissage entreprises artisanales ou de 10 salariés au plus (loi de 1979)", 'CAP', inplace=True,)
df_donnee_c['Type de contrat'].replace("Contrat de professionnalisation", 'CP', inplace=True,)
df_donnee_c['Type de contrat'] = df_donnee_c['Type de contrat'].replace("Stage",0)
df_donnee_c['Type de contrat'] = df_donnee_c['Type de contrat'].replace("CAP",1)
df_donnee_c['Type de contrat'] = df_donnee_c['Type de contrat'].replace("CP",1)
df_donnee_c['Type de contrat'] = df_donnee_c['Type de contrat'].replace("CDD",2)
df_donnee_c['Type de contrat'] = df_donnee_c['Type de contrat'].replace("CDI",3)
df_donnee_c['Véhicule'] = df_donnee_c['Véhicule'].replace("avec vehicule", 1)
df_donnee_c['Véhicule'].fillna(0,inplace = True)




#encoding meteo
df_meteo_c['Humeur'] = df_meteo_c['Humeur'].replace("Pluie", -1)
df_meteo_c['Humeur'] = df_meteo_c['Humeur'].replace("Nuage", 0)
df_meteo_c['Humeur'] = df_meteo_c['Humeur'].replace("Soleil", 1)


In [ ]:
df_donnee_c

#🟥Analyse univariate

##🟦Analyse Accidents travails

### Affichage et travail sur les données

In [ ]:
df_acc_c

In [ ]:
df_acc_c.info()

In [ ]:
# y a t'il des données manquantes ?

print(df_acc_c.isna().any())

# Oui, on veut les voir

#missing_percentage(df_acc_c)


In [ ]:
df_acc_c['AT reconnu?'].hist()

In [ ]:
df_acc_c['Localisation'].hist()

In [ ]:
### Trés peu de "non" --> suppression OU remplissage avec classe majoritaire = oui"
df_acc_c = df_acc_c.dropna(inplace=False)


In [ ]:
print(df_acc_c.isna().any())
df_acc_c.info()

## 🟦Analyse Données personnelles

### Affichage et travail sur les données

In [ ]:
df_donnee_c

In [ ]:
# y a t'il des données manquantes ?

print(df_donnee_c.isna().any())

# Oui, on veut les voir

missing_percentage(df_donnee_c)

In [ ]:
df_donnee_c['Type de contrat'].hist()

In [ ]:
df_donnee_c['Ancienneté']


On voit qu'il manque beaucoup de données pour l'ancienneté mais que pour quasiment toutes les autres la date d'ancienneté est identique à celle du début de contrat, nous allons donc remplacer l'ancienneté par une durée en année crée à partir de la date de contrat pour un chaque employé

Travail sur Ancienneté:

In [ ]:
#Changement des valeurs nuls de Ancienneté par celles de Début 1er contrat + verififcation


df_donnee_c['Ancienneté'].fillna(df_donnee_c['Début 1e contrat'],inplace = True)

assert not(df_donnee_c['Ancienneté'].isna().any())

df_donnee_c[['Ancienneté','Début 1e contrat']]

In [ ]:
# Remplacer la date d'ancienneté par une valeur en année

df_donnee_c['Ancienneté']=df_donnee_c['Ancienneté'].apply(calcul_annee)
df_donnee_c


On peut maintenant supprimer la colonne début 1er contrat

Travail sur date de naissance :

In [ ]:
# a faire ou pas ? simple

In [ ]:
# Remplacer la date de naissance par l'age

df_donnee_c['Date de naissance']=df_donnee_c['Date de naissance'].apply(calcul_annee)
df_donnee_c

In [ ]:
# on change le nom de la colonne par Age

df_donnee_c.rename(columns={"Date de naissance": "Age"}, inplace=True)
df_donnee_c

Travail sur Civilité:

In [ ]:
# on change le nom de la colonne par Sexe

df_donnee_c.rename(columns={"Civilité": "Sexe"}, inplace=True)
df_donnee_c

## 🟦Analyse Météo

### Affichage et travail sur les données

In [ ]:
df_meteo_c

In [ ]:
# y a t'il des données manquantes ?

print(df_meteo_c.isna().any())


## 🟦Analyse Absences

### Affichage et travail sur les données

In [ ]:
print(df_abs_c['Type absence'].unique())

df_abs_c

In [ ]:
# y a t'il des données manquantes ?

print(df_abs_c.isna().any())

# Oui, on veut les voir

#missing_percentage(df_abs_c)

In [ ]:
#Création d'une colonne durée absence à partir de début absence et fin absence

df_abs_c['duree_absence'] = (df_abs_c['Fin absence'] - df_abs_c['Début absence']) / np.timedelta64(1, 'D')
df_abs_c = df_abs_c[['Identifiant salarié','Début absence', 'Fin absence', 'duree_absence', 'Type absence']]

df_abs_c

## 🟦Analyse Messages envoyés

### Affichage et travail sur les données

In [ ]:
df_me_c

In [ ]:
# y a t'il des données manquantes ?

print(df_me_c.isna().any())

## 🟦Analyse Messages reçus

### Affichage et travail sur les données

In [ ]:
df_mr_c

In [ ]:
# y a t'il des données manquantes ?

print(df_mr_c.isna().any())

## 🟦Analyse Evenements

### Affichage et travail sur les données

In [ ]:
df_evenements_c

In [ ]:
# y a t'il des données manquantes ?

print(df_evenements_c.isna().any())

# Oui, on veut les voir

missing_percentage(df_evenements_c)

## 🟦Analyse Demandes

### Affichage et travail sur les données

In [ ]:
df_dmd_c

In [ ]:
# y a t'il des données manquantes ?

print(df_dmd_c.isna().any())

# Oui, on veut les voir

#missing_percentage(df_dmd_c)

## 🟦Analyse Habilitations

### Affichage et travail sur les données

In [ ]:
df_hab_c

In [ ]:
# y a t'il des données manquantes ?

print(df_hab_c.isna().any())

## 🟦Analyse Formations


### Affichage et travail sur les données

In [ ]:
df_formations_c

In [ ]:
# y a t'il des données manquantes ?

print(df_formations_c.isna().any())

# Oui, on veut les voir

missing_percentage(df_formations_c)

#🎁 Colonne numerique / categorique / unique

In [ ]:
categorical_col = ['Sexe','Type de contrat','AT reconnu?', 'Humeur']
numerical_col = ['Frequence demande', 'Frequence accident', 'Frequence message recu', 'Frequence message envoyé', 'Frequence absence', 'Age', 'Ancienneté', ]
unique_col = ['Identifiant salarié']

#🟥Analyse multivariate

On va reamenager nos dataframe en gardant que les colonnes que l'on souhaite et qui nous semble importante pour ensuite les liés.

##🟦Création d'un dataframe à partir de tous les autres

###Dataframe avec identifiant salarié unique

####Choix des features importantes pour chaque **dataframe**

In [ ]:
df_cor_c = df_acc_c.copy(deep=True)
del df_cor_c['Localisation']
del df_cor_c['Description']
del df_cor_c['AT reconnu?']
del df_cor_c['date']

In [ ]:
df_cor_donnee = df_donnee_c.copy(deep=True)
del df_cor_donnee['Début 1e contrat']
del df_cor_donnee['Situation familiale']
del df_cor_donnee['Emploi']

In [ ]:
df_cor_abs = df_abs_c.copy(deep=True)
del df_cor_abs['Type absence']

In [ ]:
df_cor_me = df_me_c.copy(deep=True)
del df_cor_me['Type de message']
del df_cor_me['Texte message']

In [ ]:
df_cor_mr = df_mr_c.copy(deep=True)
del df_cor_mr['Type de message']
del df_cor_mr['Texte message']

In [ ]:
df_cor_evt = df_evenements_c.copy(deep=True)
# utilité ?

In [ ]:
df_cor_dmd = df_dmd_c.copy(deep=True)
del df_cor_dmd['Demande']
del df_cor_dmd['Type de demande']


In [ ]:
# habilitations et formations on garde en dataframe qualitatif

####On les regroupe

In [ ]:
freq = df_cor_abs['Identifiant salarié'].value_counts().sort_index().rename_axis('Identifiant salarié').to_frame('Nombre absence')
df_cor = pd.merge(freq,df_cor_donnee,how="outer",on='Identifiant salarié')
df_cor['Nombre absence'] = df_cor['Nombre absence'].fillna(0)

In [ ]:
freq = df_cor_dmd['Identifiant salarié'].value_counts().sort_index().rename_axis('Identifiant salarié').to_frame('Nombre demande')
df_cor = pd.merge(freq,df_cor,how="outer",on='Identifiant salarié')
df_cor['Nombre demande'] = df_cor['Nombre demande'].fillna(0)

In [ ]:
freq = df_cor_me['Identifiant salarié'].value_counts().sort_index().rename_axis('Identifiant salarié').to_frame('Nombre message envoyé')
df_cor = pd.merge(freq,df_cor,how="outer",on='Identifiant salarié')
df_cor['Nombre message envoyé'] = df_cor['Nombre message envoyé'].fillna(0)

In [ ]:
freq = df_cor_mr['Identifiant salarié'].value_counts().sort_index().rename_axis('Identifiant salarié').to_frame('Nombre message recu')
df_cor = pd.merge(freq,df_cor,how="outer",on='Identifiant salarié')
df_cor['Nombre message recu'] = df_cor['Nombre message recu'].fillna(0)

In [ ]:
freq = df_cor_c['Identifiant salarié'].value_counts().sort_index().rename_axis('Identifiant salarié').to_frame('Nombre accident')
df_cor = pd.merge(freq,df_cor,how="outer",on='Identifiant salarié')
df_cor['Nombre accident'] = df_cor['Nombre accident'].fillna(0)

Nous avons un dataframe complet mais il reste quelques modifications

In [ ]:
# y a t'il des données manquantes ?

print(df_cor.isna().any())

# Oui, on veut les voir

#missing_percentage(df_cor)

Il faut changer les nombres de demandes, nombres de messages etc par une frequence en fonction de l'ancienneté de la personne sinon ca n'a pas de sens.

In [ ]:
df_cor['Nombre absence'] = (df_cor['Nombre absence']/df_cor['Ancienneté'])
df_cor.rename(columns={'Nombre absence': 'Frequence absence'},inplace=True)

df_cor['Nombre message recu'] = (df_cor['Nombre message recu']/df_cor['Ancienneté'])
df_cor.rename(columns={'Nombre message recu': 'Frequence message recu'},inplace= True)

df_cor['Nombre message envoyé'] = (df_cor['Nombre message envoyé']/df_cor['Ancienneté'])
df_cor.rename(columns={'Nombre message envoyé': 'Frequence message envoyé'},inplace=True)

df_cor['Nombre accident'] = (df_cor['Nombre accident']/df_cor['Ancienneté'])
df_cor.rename(columns={'Nombre accident': 'Frequence accident'},inplace=True)

df_cor['Nombre demande'] = (df_cor['Nombre demande']/df_cor['Ancienneté'])
df_cor.rename(columns={'Nombre demande': 'Frequence demande'},inplace=True)
df_cor


###Dataframe pour les times series

####Choix des features importantes pour chaque **dataframe**

In [ ]:
df_cor2_dmd = df_dmd_c.copy(deep=True)


In [ ]:
df_cor2_evt = df_evenements_c.copy(deep=True)
# utilité ?

In [ ]:
df_cor2_mr = df_mr_c.copy(deep=True)
del df_cor2_mr['Type de message']
del df_cor2_mr['Texte message']

In [ ]:
df_cor2_me = df_me_c.copy(deep=True)
del df_cor2_me['Type de message']
del df_cor2_me['Texte message']

In [ ]:
df_cor2_abs = df_abs_c.copy(deep=True)
del df_cor2_abs['Type absence']

In [ ]:
df_cor2_meteo = df_meteo_c.copy(deep=True)

df_cor2_meteo

In [ ]:
df_cor2_donnee = df_donnee_c.copy(deep=True)
#del df_cor2_donnee['Début 1e contrat']
del df_cor2_donnee['Situation familiale']
del df_cor2_donnee['Emploi']
df_cor2_donnee

In [ ]:
df_cor2_c = df_acc_c.copy(deep=True)
#df_cor2_c = df_cor2_c.set_index('date')
df_trie = df_cor2_c.sort_values(by='date', ascending=False)
df_trie

####Création des vecteurs

In [ ]:
# Création d'un tableau vide avec toutes les dates entre 2008 et 2017
date_range_2008_2017 = pd.date_range(start='2008-01-01', end='2017-12-31', freq='2W')
all_dates_2008_2017 = pd.DataFrame(date_range_2008_2017, columns=['Start_Date'])

# Création d'un tableau vide avec toutes les dates entre 2018 et 2023
date_range_2018_2023 = pd.date_range(start='2018-01-01', end='2023-12-31', freq='2W')
all_dates_2018_2023 = pd.DataFrame(date_range_2018_2023, columns=['Start_Date'])

# Créer un dictionnaire pour stocker les vecteurs d'accidents pour les années 2008 à 2017 et 2018 à 2023
accident_vectors_2008_2017 = {}
accident_vectors_2018_2023 = {}

# Itération sur chaque identifiant de salarié
for id_salarie in df_cor2_donnee['Identifiant salarié'].unique():
    # Filtrer les données pour un identifiant de salarié donné
    df_filtered = df_cor2_c[df_cor2_c['Identifiant salarié'] == id_salarie]

    # Créer un vecteur pour stocker les accidents pour les années 2008 à 2017
    accident_vector_2008_2017 = np.zeros(len(all_dates_2008_2017), dtype=int)

    # Itération sur chaque période de deux semaines pour les années 2008 à 2017
    for i, start_date in enumerate(all_dates_2008_2017['Start_Date']):
        end_date = start_date + pd.DateOffset(weeks=2)
        accidents_in_period = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] < end_date)]
        accident_vector_2008_2017[i] = len(accidents_in_period)

    # Stocker le vecteur d'accidents pour les années 2008 à 2017 dans le dictionnaire
    accident_vectors_2008_2017[id_salarie] = accident_vector_2008_2017

    # Créer un vecteur pour stocker les accidents pour les années 2018 à 2023
    accident_vector_2018_2023 = np.zeros(len(all_dates_2018_2023), dtype=int)

    # Itération sur chaque période de deux semaines pour les années 2018 à 2023
    for i, start_date in enumerate(all_dates_2018_2023['Start_Date']):
        end_date = start_date + pd.DateOffset(weeks=2)
        accidents_in_period = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] < end_date)]
        accident_vector_2018_2023[i] = len(accidents_in_period)

    # Stocker le vecteur d'accidents pour les années 2018 à 2023 dans le dictionnaire
    accident_vectors_2018_2023[id_salarie] = accident_vector_2018_2023


# # Afficher les vecteurs d'accidents pour chaque salarié pour les années 2008 à 2017
# print("Vecteurs d'accidents pour les années 2008 à 2017 :")
# for id_salarie, vector in accident_vectors_2008_2017.items():
#     print(f"Identifiant salarié: {id_salarie}, Vecteur d'accidents: {vector}")

# Afficher les vecteurs d'accidents pour chaque salarié pour les années 2018 à 2023
# print("\nVecteurs d'accidents pour les années 2018 à 2023 :")
# for id_salarie, vector in accident_vectors_2018_2023.items():
#     print(f"Identifiant salarié: {id_salarie}, Vecteur d'accidents: {vector}")


In [ ]:
# Itération sur chaque identifiant de salarié
for id_salarie in df_cor2_c['Identifiant salarié'].unique():
    # Filtrer les données pour un identifiant de salarié donné
    df_filtered = df_cor2_c[df_cor2_c['Identifiant salarié'] == id_salarie]

    # Récupérer la date de début du premier contrat du salarié
    debut_contrat = df_cor2_donnee[df_cor2_donnee['Identifiant salarié'] == id_salarie]['Début 1e contrat'].iloc[0]
    print(f"Date de début du contrat pour le salarié {id_salarie} : {debut_contrat}")

    # Convertir les dates en objets datetime pour la comparaison
    debut_contrat_datetime = debut_contrat.to_pydatetime()
    print(f"Type de debut_contrat_datetime : {type(debut_contrat_datetime)}")
    print(f"Type de datetime(2018, 1, 1) : {type(datetime(2018, 1, 1))}")
    print(f"Type de datetime(2023, 12, 31) : {type(datetime(2023, 12, 31))}")

    # Vérifier si la date de début de contrat est dans la plage 2018-2023
    if datetime(2018, 1, 1) <= debut_contrat_datetime and debut_contrat_datetime <= datetime(2023, 12, 31):
        # Trouver l'indice de début de contrat dans les dates 2018-2023
        matching_dates = all_dates_2018_2023[all_dates_2018_2023['Start_Date'] == debut_contrat]
        print(f"Traitement du salarié avec l'identifiant : {id_salarie}")
        if not matching_dates.empty:
            start_index = matching_dates.index[0]

            # Créer un vecteur pour stocker les accidents pour les années 2018 à 2023
            accident_vector_2018_2023 = np.zeros(len(all_dates_2018_2023), dtype=int)

            # Copier les valeurs du vecteur de la date de début de contrat jusqu'en 2023
            accident_vector_2018_2023[:start_index] = accident_vectors_2008_2017[id_salarie][-start_index:]

            # Itération sur chaque période de deux semaines pour les années 2018 à 2023
            for i, start_date in enumerate(all_dates_2018_2023['Start_Date'][start_index:]):
                end_date = start_date + pd.DateOffset(weeks=2)
                accidents_in_period = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] < end_date)]
                accident_vector_2018_2023[i + start_index] = len(accidents_in_period)

            # Stocker le vecteur d'accidents pour les années 2018 à 2023 dans le dictionnaire
            accident_vectors_2018_2023[id_salarie] = accident_vector_2018_2023
        else:
            print(f"Ignorer le salarié {id_salarie} car sa date de début de contrat n'est pas trouvée dans les dates 2018-2023.")
    else:
        # Si la date de début de contrat n'est pas dans la plage 2018-2023, ignorer ce salarié
        print(f"Ignorer le salarié {id_salarie} car sa date de début de contrat n'est pas dans la plage 2018-2023.")


In [ ]:
debut_contrat= df_cor2_donnee[df_cor2_donnee['Identifiant salarié'] == 346]['Début 1e contrat'].iloc[0]
matching_dates = all_dates_2018_2023[all_dates_2018_2023['Start_Date'] == debut_contrat]
matching_dates

In [ ]:
humeur_vectors_2008_2017 = {}
humeur_vectors_2018_2023 = {}
# Itération sur chaque identifiant de salarié
for id_salarie in df_cor2_meteo['Identifiant salarié'].unique():
    df_filtered = df_cor2_meteo[df_cor2_meteo['Identifiant salarié'] == id_salarie]

    # Créer un vecteur pour stocker les moyennes de 'humeur' pour les années 2008 à 2017
    humeur_vector_2008_2017 = np.zeros(len(all_dates_2008_2017), dtype=float)

    # Itération sur chaque période de deux semaines pour les années 2008 à 2017
    for i, start_date in enumerate(all_dates_2008_2017['Start_Date']):
        end_date = start_date + pd.DateOffset(weeks=2)
        df_period = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] < end_date)]
        if not df_period.empty:
            humeur_vector_2008_2017[i] = pd.to_numeric(df_period['Humeur'], errors='coerce').mean()

    # Stocker le vecteur de moyennes de 'humeur' pour les années 2008 à 2017 dans le dictionnaire
    humeur_vectors_2008_2017[id_salarie] = humeur_vector_2008_2017

    # Créer un vecteur pour stocker les moyennes de 'humeur' pour les années 2018 à 2023
    humeur_vector_2018_2023 = np.zeros(len(all_dates_2018_2023), dtype=float)

    # Itération sur chaque période de deux semaines pour les années 2018 à 2023
    for i, start_date in enumerate(all_dates_2018_2023['Start_Date']):
        end_date = start_date + pd.DateOffset(weeks=2)
        df_period = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] < end_date)]
        if not df_period.empty:
            humeur_vector_2018_2023[i] = pd.to_numeric(df_period['Humeur'], errors='coerce').mean()

    # Stocker le vecteur de moyennes de 'humeur' pour les années 2018 à 2023 dans le dictionnaire
    humeur_vectors_2018_2023[id_salarie] = humeur_vector_2018_2023

# # Afficher les vecteurs de moyennes de 'humeur' pour chaque salarié pour les années 2008 à 2017
# print("Vecteurs de moyennes de 'humeur' pour les années 2008 à 2017 :")
# for id_salarie, vector in humeur_vectors_2008_2017.items():
#     print(f"Identifiant salarié: {id_salarie}, Vecteur de moyennes de 'humeur': {vector}")

# # Afficher les vecteurs de moyennes de 'humeur' pour chaque salarié pour les années 2018 à 2023
# print("\nVecteurs de moyennes de 'humeur' pour les années 2018 à 2023 :")
# for id_salarie, vector in humeur_vectors_2018_2023.items():
#     print(f"Identifiant salarié: {id_salarie}, Vecteur de moyennes de 'humeur': {vector}")


In [ ]:
humeur_vectors_2018_2023[1530]

In [ ]:
no_meteo = [id_salarie for id_salarie in df_cor2_donnee['Identifiant salarié'].unique() if id_salarie not in df_cor2_meteo['Identifiant salarié'].unique()]

no_accident = [id_salarie for id_salarie in df_cor2_donnee['Identifiant salarié'].unique() if id_salarie not in df_cor2_c['Identifiant salarié'].unique()]

print('Pas de donnée humeur :', no_meteo)
print('Pas de donnée accident :', no_accident)

In [ ]:
# Itération sur chaque identifiant de salarié
for id_salarie in df_cor2_meteo['Identifiant salarié'].unique():
    # Récupérer le vecteur de l'humeur correspondant à l'identifiant du salarié
    humeur_vector = humeur_vectors_2018_2023.get(id_salarie)
    accident_vector = accident_vectors_2018_2023.get(id_salarie)
    df_cor.loc[df_cor['Identifiant salarié'] == id_salarie, 'vecteur_humeur'] = [humeur_vector]
    df_cor.loc[df_cor['Identifiant salarié'] == id_salarie, 'vecteur_accident'] = [accident_vector]
##testeter avec humeur_vectors_2018_2023['id_salarie']

#     if humeur_vector is not None:
#         # Convertir le vecteur en une chaîne JSON pour stockage
#         humeur_vector_json = json.dumps(humeur_vector.tolist())
#         accident_vector_json = json.dumps(accident_vector.tolist())
#         # Mettre à jour la ligne correspondante dans df_cor avec les vecteurs
#         df_cor.loc[df_cor['Identifiant salarié'] == id_salarie, 'vecteur_humeur'] = humeur_vector_json
#         df_cor.loc[df_cor['Identifiant salarié'] == id_salarie, 'vecteur_accident'] = accident_vector_json


# df_cor
df_cor

##🟦Clustering

###Clustering sans times series

####ACP sur tout

In [ ]:
# Sélectionner les caractéristiques pertinentes
features = df_cor[['Frequence accident', 'Frequence message recu', 'Frequence message envoyé', 'Frequence demande', 'Frequence absence', 'Ancienneté', 'Type de contrat',]]

# Normaliser les données
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Appliquer l'analyse en composantes principales
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

# Appliquer l'algorithme K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(features_pca)

# Affichage graphique des clusters
plt.figure(figsize=(10, 6))

# Tracer les points de données pour chaque cluster
for cluster_label in range(3):
    plt.scatter(features_pca[kmeans.labels_ == cluster_label][:, 0],
                features_pca[kmeans.labels_ == cluster_label][:, 1],
                label=f'Cluster {cluster_label + 1}')

# Tracer les centres de chaque cluster
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], marker='x', color='black', label='Cluster Center')

plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.title('K-Means avec une ACP')
plt.legend()
plt.show()

In [ ]:
# Trouver l'indice du centroïde le plus proche de chaque centre de cluster
centroid_indices = []
for centroid in kmeans.cluster_centers_:
    distances = cdist(features_pca, [centroid])
    centroid_index = np.argmin(distances)
    centroid_indices.append(centroid_index)

# Récupérer les exemples pertinents pour chaque cluster
relevant_examples = df_cor.iloc[centroid_indices]

# Afficher les exemples pertinents pour chaque cluster
print(relevant_examples)

####k-means abs accidentologie et ?

In [ ]:
# Sélectionner les caractéristiques pertinentes
features = df_cor[['Frequence accident', 'Frequence message recu', 'Frequence message envoyé', 'Frequence demande', 'Frequence absence', 'Ancienneté', 'Type de contrat', ]]

# Normaliser les données
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Choisir les axes x et y (par exemple, 'Frequence accident' et 'Ancienneté')
x_axis = 'Frequence accident'
y_axis = 'Ancienneté'

# Utiliser seulement les colonnes sélectionnées pour l'algorithme K-Means
selected_features = features[[x_axis, y_axis]]

# Appliquer l'algorithme K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(selected_features)

# Affichage graphique des clusters
plt.figure(figsize=(10, 6))

# Tracer les points de données pour chaque cluster
for cluster_label in range(3):
    plt.scatter(selected_features[kmeans.labels_ == cluster_label][x_axis],
                selected_features[kmeans.labels_ == cluster_label][y_axis],
                label=f'Cluster {cluster_label + 1}')

# Tracer les centres de chaque cluster
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], marker='x', color='black', label='Cluster Center')

plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.title('K-Means sans ACP')
plt.legend()
plt.show()

On essaie d'appliquer dbscan plutot que k-means pour voir la différence car le résultat est peu convaincant et ressemble à une forme faisant penser qu'il faudrait utiliser dbscan

In [ ]:
# Sélectionner les caractéristiques pertinentes
features = df_cor[['Frequence accident', 'Frequence message recu', 'Frequence message envoyé', 'Frequence demande', 'Frequence absence', 'Ancienneté', 'Type de contrat']]

# Normaliser les données
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Choisir les axes x et y (par exemple, 'Frequence accident' et 'Ancienneté')
x_axis = 'Frequence accident'
y_axis = 'Ancienneté'

# Utiliser seulement les colonnes sélectionnées pour l'algorithme DBSCAN
selected_features = features[[x_axis, y_axis]]

# Appliquer l'algorithme DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Vous pouvez ajuster les paramètres eps et min_samples
dbscan_labels = dbscan.fit_predict(selected_features)

# Affichage graphique des clusters
plt.figure(figsize=(10, 6))

# Tracer les points de données pour chaque cluster (y compris le bruit, labelé -1 par DBSCAN)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        plt.scatter(selected_features[dbscan_labels == cluster_label][x_axis],
                    selected_features[dbscan_labels == cluster_label][y_axis],
                    label=f'Noise')
    else:
        plt.scatter(selected_features[dbscan_labels == cluster_label][x_axis],
                    selected_features[dbscan_labels == cluster_label][y_axis],
                    label=f'Cluster {cluster_label + 1}')

plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.title('DBSCAN sans ACP')
plt.legend()
plt.show()


#### ACP sur tout - accident + 3d avec accident


In [ ]:
# Sélectionner les caractéristiques pertinentes
features = df_cor[['Frequence accident', 'Frequence message recu', 'Frequence message envoyé', 'Frequence demande', 'Frequence absence', 'Ancienneté', 'Type de contrat']]

# Normaliser les données
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Appliquer l'analyse en composantes principales
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

# Ajouter la fréquence d'accident en tant que troisième dimension
features_3d = pd.concat([pd.DataFrame(features_pca, columns=['PC1', 'PC2']), features['Frequence accident']], axis=1)

# Appliquer l'algorithme K-Means
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans.fit_predict(features_3d)

# Affichage graphique en trois dimensions avec K-Means
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Tracer les points de données pour chaque cluster
for cluster_label in range(4):
    ax.scatter(features_3d.loc[kmeans_labels == cluster_label, 'PC1'],
               features_3d.loc[kmeans_labels == cluster_label, 'PC2'],
               features_3d.loc[kmeans_labels == cluster_label, 'Frequence accident'],
               label=f'Cluster {cluster_label + 1}')

# Tracer les centres de chaque cluster
ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], kmeans.cluster_centers_[:, 2],
           marker='x', color='black', label='Cluster Center')

ax.set_xlabel('Composante principale 1')
ax.set_ylabel('Composante principale 2')
ax.set_zlabel('Frequence accident')
ax.set_title('K-Means avec ACP en 3D')
ax.legend()
plt.show()


In [ ]:
# Trouver l'indice du centroïde le plus proche de chaque centre de cluster
centroid_indices = []
for centroid in kmeans.cluster_centers_:
    distances = cdist(features_3d[['PC1', 'PC2', 'Frequence accident']], [centroid])
    centroid_index = np.argmin(distances)
    centroid_indices.append(centroid_index)

# Récupérer les exemples pertinents pour chaque cluster
relevant_examples = df_cor.iloc[centroid_indices]

# Afficher les exemples pertinents pour chaque cluster
print(relevant_examples)

###Clustering avec times series

##🔎Premiere analyse à l'aide de sweetviz

In [ ]:
!pip install sweetviz

In [ ]:
import sweetviz as sv

In [ ]:
report_bivariate_fo = sv.analyze(source=df_cor,
                                 target_feat='Frequence accident', # add focus on our target value
                                 feat_cfg=None,
                                 pairwise_analysis='on') # let's see the relation btw our variables
report_bivariate_fo.show_html(filepath="reports.html")
report_bivariate_fo.show_notebook(w=2000, h=1000)

##🔎Corrélation

###Corrélation avec identifiant salarié unique

In [ ]:
plot_correlation_with_feature_v2(df_cor,'Frequence accident')